In [106]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split as tts
import re
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical



In [107]:
df=pd.read_csv("Senti.csv")

In [108]:
df

,id,text,sentiment
0,1,I love this car,Positive
1,2,This view is amazing,Positive
2,3,I feel great this morning,Positive
3,4,I am so excited about the concert,Positive
4,5,He is my best friend,Positive
5,6,I do not like this car,Negative
6,7,This view is horrible,Negative
7,8,I feel tired this morning,Negative
8,9,I am not looking forward to the concert,Negative
9,10,He is my enemy,Negative


In [109]:
df.shape

(10, 3)

In [110]:
import re

df = df[df.sentiment != "Neutral"]
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x)) # removing all the special character form the txt 

In [111]:
df['text']

0                            i love this car
1                       this view is amazing
2                  i feel great this morning
3          i am so excited about the concert
4                       he is my best friend
5                     i do not like this car
6                      this view is horrible
7                  i feel tired this morning
8    i am not looking forward to the concert
9                             he is my enemy
Name: text, dtype: object

In [112]:
df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('rt', ' ').astype(int)

In [113]:
max_feature=2000
tokenizer=Tokenizer(nb_words=max_feature,split=' ')
print(tokenizer)

c:\Users\somes\anaconda3\Lib\site-packages\keras\src\legacy\preprocessing\text.py:98: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


In [132]:
tokenizer.fit_on_texts(df['text'].values)


In [133]:
x=tokenizer.texts_to_sequences(df['text'].values)
x=pad_sequences(x)

In [115]:
print(x)

[[ 0  0  0  0  1 14  2  4]
 [ 0  0  0  0  2  5  3 15]
 [ 0  0  0  1  6 16  2  7]
 [ 0  1  8 17 18 19  9 10]
 [ 0  0  0 11  3 12 20 21]
 [ 0  0  1 22 13 23  2  4]
 [ 0  0  0  0  2  5  3 24]
 [ 0  0  0  1  6 25  2  7]
 [ 1  8 13 26 27 28  9 10]
 [ 0  0  0  0 11  3 12 29]]


In [116]:
y= pd.get_dummies(df['sentiment']).values

In [117]:
y

array([[False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False]])

In [118]:
x_train,x_test,y_train,y_test=tts(x,y,test_size=0.3,random_state=42)
print('shape of training samples',x_train.shape,y_train.shape)
print('shape of testing sample',x_test.shape,y_test.shape) 

shape of training samples (7, 8) (7, 2)
shape of testing sample (3, 8) (3, 2)


In [119]:
x_train[1]

array([ 0,  0,  0,  1,  6, 25,  2,  7])

In [120]:
model=Sequential()
model.add(Embedding(input_dim=max_feature, output_dim=128)) 
model.add(LSTM(128))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [121]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [122]:
model

<Sequential name=sequential_7, built=False>

In [123]:
model.fit(x_train,y_train,epochs=10,batch_size=128,verbose=2,validation_data=(x_test,y_test))

Epoch 1/10
1/1 - 2s - 2s/step - accuracy: 0.2857 - loss: 0.6926 - val_accuracy: 0.0000e+00 - val_loss: 0.7051
Epoch 2/10
1/1 - 0s - 66ms/step - accuracy: 0.8571 - loss: 0.6839 - val_accuracy: 0.3333 - val_loss: 0.7174
Epoch 3/10
1/1 - 0s - 59ms/step - accuracy: 0.8571 - loss: 0.6750 - val_accuracy: 0.3333 - val_loss: 0.7309
Epoch 4/10
1/1 - 0s - 64ms/step - accuracy: 0.8571 - loss: 0.6655 - val_accuracy: 0.3333 - val_loss: 0.7464
Epoch 5/10
1/1 - 0s - 61ms/step - accuracy: 0.8571 - loss: 0.6553 - val_accuracy: 0.3333 - val_loss: 0.7643
Epoch 6/10
1/1 - 0s - 54ms/step - accuracy: 0.8571 - loss: 0.6441 - val_accuracy: 0.3333 - val_loss: 0.7857
Epoch 7/10
1/1 - 0s - 64ms/step - accuracy: 0.8571 - loss: 0.6317 - val_accuracy: 0.0000e+00 - val_loss: 0.8113
Epoch 8/10
1/1 - 0s - 59ms/step - accuracy: 0.8571 - loss: 0.6178 - val_accuracy: 0.0000e+00 - val_loss: 0.8426
Epoch 9/10
1/1 - 0s - 58ms/step - accuracy: 0.8571 - loss: 0.6022 - val_accuracy: 0.0000e+00 - val_loss: 0.8810
Epoch 10/10
1/

## VALIDATION

In [124]:
score,acc=model.evaluate(x_test,y_test)
print(score)
print(acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0000e+00 - loss: 0.9284
0.9284446835517883
0.0


## SAMPLE TESTING

In [125]:
text='I am do not like him'
tester=np.array([text])
tester=pd.DataFrame(tester)
tester.columns=['text']
tester['text'] = tester['text'].apply(lambda x: x.lower())
tester['text'] = tester['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [126]:
print(tester)

                   text
0  i am do not like him


In [127]:
max_feature=2000
test=tokenizer.texts_to_sequences(tester['text'].values)
test=pad_sequences(x)

In [128]:
test

array([[ 0,  0,  0,  0,  1, 14,  2,  4],
       [ 0,  0,  0,  0,  2,  5,  3, 15],
       [ 0,  0,  0,  1,  6, 16,  2,  7],
       [ 0,  1,  8, 17, 18, 19,  9, 10],
       [ 0,  0,  0, 11,  3, 12, 20, 21],
       [ 0,  0,  1, 22, 13, 23,  2,  4],
       [ 0,  0,  0,  0,  2,  5,  3, 24],
       [ 0,  0,  0,  1,  6, 25,  2,  7],
       [ 1,  8, 13, 26, 27, 28,  9, 10],
       [ 0,  0,  0,  0, 11,  3, 12, 29]])

In [129]:
if x.shape[1] > test.shape[1]:
    test=np.pad(test[0],(x.shape[1]-test.shape[1],0,'constant'))
    

In [130]:
test

array([[ 0,  0,  0,  0,  1, 14,  2,  4],
       [ 0,  0,  0,  0,  2,  5,  3, 15],
       [ 0,  0,  0,  1,  6, 16,  2,  7],
       [ 0,  1,  8, 17, 18, 19,  9, 10],
       [ 0,  0,  0, 11,  3, 12, 20, 21],
       [ 0,  0,  1, 22, 13, 23,  2,  4],
       [ 0,  0,  0,  0,  2,  5,  3, 24],
       [ 0,  0,  0,  1,  6, 25,  2,  7],
       [ 1,  8, 13, 26, 27, 28,  9, 10],
       [ 0,  0,  0,  0, 11,  3, 12, 29]])

In [131]:
prediciton=model.predict(test)
prediciton[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


array([0.377089  , 0.62291104], dtype=float32)